# Rasgos

Las posibilidades que ofrece el despacho múltiple en combinación con la riqueza ilimitada del sistema de tipos son enormes.

Uno de los patrones más comunes en el desarrollo de código en Julia es el de los **Tim Holy traits** o **rasgos**, nombrado así en honor a su [descubridor](https://github.com/JuliaLang/julia/issues/2345#issuecomment-54537633) (¿o creador...?).

Los rasgos representan comportamientos que un tipo puede tener. En un ejemplo como el del Notebook anterior, tanto un `Aguila` como un `Murcielago` pueden volar y caminar. Un pingüino puede caminar y nadar, pero no puede volar. En cambio, un `Pato` puede volar, nadar y caminar. Típicamente, los rasgos son binarios: un tipo puede tenerlo o no tenerlo.

Los rasgos ofrecen una forma de extender y complejizar el sistema de tipos y sus comportamientos sin tener que modificar la base.  

Definamos nuevamente los tipos del Notebook anterior: 

In [2]:
abstract type Animal end

abstract type Mamifero <: Animal end
abstract type Ave <: Animal end
abstract type Reptil <: Animal end
abstract type Anfibio <: Animal end
abstract type Insecto <: Animal end

In [11]:
mutable struct Aguila <: Ave
    x::Float64
    y::Float64
    z::Float64
end

mutable struct Murcielago <: Mamifero
    x::Float64
    y::Float64
    z::Float64
end

mutable struct Pinguino <: Ave
    x::Float64
    y::Float64
end

Supongamos que queremos definir una función

```julia
volar!(a::Animal, pos::Vector{Float64})
```

que lleve al animal `a` volando a la nueva posición `pos`, si es que el animal puede volar

La solución más rudimentaria sería

In [1]:
puede_volar(a::Animal) = false # Por defecto, no puede volar
puede_volar(a::Aguila) = true

function volar_if!(a::Animal, pos::Vector{Float64})
    if puede_volar(animal)
        animal.x = pos[1]
        animal.y = pos[2]
        println("Volando a ", pos)
    else
        error("No puede volar")
    end
end

UndefVarError: UndefVarError: `Animal` not defined

Sin embargo, esta solución no es buena porque implica un `if` que se resuelve en tiempo de ejecución, cuando queremos que la función compile a una versión lo más específica, y por lo tanto rápida, posible.

Lo mejor es usar los tipos para hacer despacho. Otra posibilidad es usar un tipo `Union` para despachar la funcion `volar!` al método específico.

In [14]:
function volar_Union!(a::Union{Aguila, Murcielago}, pos::Vector{Float64})
    animal.x = pos[1]
    animal.y = pos[2]
    println("Volando a ", pos)
end

function volar_Union!(a::Animal, pos::Vector{Float64})
    error("No puede volar")
end

volar_union (generic function with 2 methods)

Esta solución es mejor que la anterior, pero no es lo mejor porque para cada nuevo tipo de animal que queramos agregar, debemos acordarnos de agregarlo a la `Union` y modificar la función `volar!`.

La solución óptima es usar rasgos. Los definimos de la siguiente manera:

In [5]:
abstract type CapacidadDeVuelo end

struct PuedeVolar <: CapacidadDeVuelo end
struct NoPuedeVolar <: CapacidadDeVuelo end

y asignamos los rasgos a los distintos tipos

In [ ]:
capacidad_de_vuelo(a::Animal) = NoPuedeVolar() # Por defecto, los animales no pueden volar
capacidad_de_vuelo(a::Aguila) = PuedeVolar()

Ahora, podemos definir la función `volar` así:

In [2]:
volar!(a::Animal, pos::Vector{Float64}) = volar!(capacidad_de_vuelo(a), a, pos)

function volar!(::PuedeVolar, animal::Animal, pos::Vector{Float64})
    animal.x = pos[1]
    animal.y = pos[2]
    println("Volando a ", pos)
end

function volar!(::NoPuedeVolar, animal::Animal, pos::Vector{Float64})
    error("No puede volar")
end

UndefVarError: UndefVarError: `Animal` not defined

Ahora, cuando queremos agregar un nuevo tipo de animal volador, simplemente definimos un nuevo tipo y le asignamos el rasgo `Volador`. No necesitamos modificar el tipo Union ni la función `volar`.

In [16]:
capacidad_de_vuelo(a::Murcielago) = PuedeVolar()

capacidad_de_vuelo (generic function with 1 method)

Esto es especialmente ventajoso cuando desarrollamos código para paquetes que pueden ser usados por otros usuarios. Si queremos que los usuarios puedan agregar sus propios tipos de animales, el tipo `Union` no está disponible para extensión, mientras que con el uso de rasgos solo necesitan asignarle los rasgos correspondientes.